In [ ]:
pip install html_to_json

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=d3e3a7e445ef0b838e85ae527569d8177c3d90b72ba6588349f569af1500f7b4
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import requests, json, csv
import html_to_json
import pandas as pd

In [ ]:
'''
  Load information and get to column level
'''
pageid=input('What is the Page ID? ')
payload = requests.get('https://en.wikipedia.org/w/api.php?action=parse&format=json&pageid=' + pageid).json()['parse']['text']['*']
#test 1: 57396344 (Botanists)
#test 2: 65609442 (African-American Suffragists; shorter; takes less time to process)
#Example for LD4: 61615302 (Rugby League)
len(payload)
dict_payload=html_to_json.convert(payload)['div'][0]['table'][-1]['tbody'][0]['tr']
print(dict_payload)



What is the Page ID? 61615302
[{'th': [{'_value': 'name'}, {'_value': 'image'}, {'_value': 'description'}, {'_value': 'country of citizenship'}, {'_value': 'date of birth'}, {'_value': 'date of death'}, {'_value': 'place of birth'}, {'_value': 'place of death'}, {'_value': 'item'}, {'_value': 'site links'}]}, {'td': [{'a': [{'_attributes': {'href': '/w/index.php?title=Elisa_Ciria&action=edit&redlink=1', 'class': ['new'], 'title': 'Elisa Ciria (page does not exist)'}, '_value': 'Elisa Ciria'}]}, {}, {'style': [{'_attributes': {'data-mw-deduplicate': 'TemplateStyles:r1066479718'}, '_value': '.mw-parser-output .infobox-subbox{padding:0;border:none;margin:-3px;width:auto;min-width:100%;font-size:100%;clear:none;float:none;background-color:transparent}.mw-parser-output .infobox-3cols-child{margin:auto}.mw-parser-output .infobox .navbar{font-size:100%}body.skin-minerva .mw-parser-output .infobox-header,body.skin-minerva .mw-parser-output .infobox-subheader,body.skin-minerva .mw-parser-output

In [ ]:
'''
  Create list of names in the table with corresponding Wikidata identifier
'''
person_list = []
row_count = 0
for row in dict_payload:
  if row.get('td'):
    step_count = 0
    for step in row['td']:
      step_count += 1
      column= step.get('a')
      if type(column) == list:
        if column[0].get('_attributes').get('class') == ['new']:
          item = dict_payload[row_count]['td']
          for value in item:
            if value.get('a'):
              if value['a'][0]['_attributes']['title'].startswith('d:Q'):
                person_list.append({
                    'name':column[0].get('_value'),
                    'wikidata_code':value['a'][0]['_value'],
                    'wikidata_link':'https://wikidata.org/wiki/'+value['a'][0]['_value']
                  })
                break
  row_count += 1
print(person_list)


[{'name': 'Elisa Ciria', 'wikidata_code': 'Q108038804', 'wikidata_link': 'https://wikidata.org/wiki/Q108038804'}, {'name': 'Cristina Song-Puche', 'wikidata_code': 'Q63145797', 'wikidata_link': 'https://wikidata.org/wiki/Q63145797'}, {'name': 'Leila Bessahli', 'wikidata_code': 'Q63145664', 'wikidata_link': 'https://wikidata.org/wiki/Q63145664'}, {'name': 'Page McGregor', 'wikidata_code': 'Q109317782', 'wikidata_link': 'https://wikidata.org/wiki/Q109317782'}, {'name': 'Lauréane Biville', 'wikidata_code': 'Q63145696', 'wikidata_link': 'https://wikidata.org/wiki/Q63145696'}, {'name': 'Maïlys Borak', 'wikidata_code': 'Q119555256', 'wikidata_link': 'https://wikidata.org/wiki/Q119555256'}, {'name': 'Neta Peau', 'wikidata_code': 'Q112249476', 'wikidata_link': 'https://wikidata.org/wiki/Q112249476'}, {'name': 'Jill Adams', 'wikidata_code': 'Q116462534', 'wikidata_link': 'https://wikidata.org/wiki/Q116462534'}, {'name': 'Jane Banks', 'wikidata_code': 'Q116475202', 'wikidata_link': 'https://wikid

In [ ]:
'''
  Search for possible matches and add to dict
'''
endpoint = 'https://en.wikipedia.org/w/api.php?action=opensearch&search='
items_in_list = len(person_list)
list_with_match=[]
for person in person_list:
  items_in_list -=1
  possible_matches_to_check = []
  searchable_name = '%27' + person['name'].replace(' ', '%20',) + '%27'
  possible_matches=requests.get(endpoint+searchable_name).json()[1]
  if possible_matches:
    for possible_match in possible_matches:
      possible_match = possible_match.replace(' ', '_')
      possible_matches_to_check.append('http://en.wikipedia.org/wiki/'+ possible_match)
      person['possible_matches']=possible_matches_to_check
    list_with_match.append(person)
    print(f'Match(es) found for {person["name"]}. {items_in_list} items left to search')
  else:
    person_list.remove(person)
    print(f'No match found for {person["name"]}. {items_in_list} items left to search')


No match found for Elisa Ciria. 33 items left to search
No match found for Leila Bessahli. 32 items left to search
No match found for Lauréane Biville. 31 items left to search
No match found for Neta Peau. 30 items left to search
Match(es) found for Jane Banks. 29 items left to search
No match found for Karen Burrows. 28 items left to search
Match(es) found for Adriana Felix. 27 items left to search
Match(es) found for Francesca Goldthorp. 26 items left to search
Match(es) found for Georgia Hale. 25 items left to search
No match found for Lavinia Kitai. 24 items left to search
No match found for Joan Kuman. 23 items left to search
No match found for Giovanna Moura. 22 items left to search
No match found for Ada Okonkwo. 21 items left to search
No match found for Josefa Rasiga. 20 items left to search
Match(es) found for Karen Shaw. 19 items left to search
Match(es) found for Emma Slowe. 18 items left to search
No match found for Karen Stuart. 17 items left to search
No match found for 

In [ ]:
'''print to csv'''
header = ['name', 'wikidata_code', 'wikidata_link', 'possible_matches']
csv_file = 'test.csv'
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=header)
        writer.writeheader()
        for data in list_with_match:
            writer.writerow(data)
except IOError:
    print("I/O error")
df1=pd.read_csv(csv_file)
df1

,name,wikidata_code,wikidata_link,possible_matches
0,Jane Banks,Q116475202,https://wikidata.org/wiki/Q116475202,['http://en.wikipedia.org/wiki/June_Banks']
1,Adriana Felix,Q116492978,https://wikidata.org/wiki/Q116492978,['http://en.wikipedia.org/wiki/Adriano_Félix_T...
2,Francesca Goldthorp,Q116494457,https://wikidata.org/wiki/Q116494457,['http://en.wikipedia.org/wiki/Francesca_Goldt...
3,Georgia Hale,Q116495194,https://wikidata.org/wiki/Q116495194,"['http://en.wikipedia.org/wiki/Georgia_Hale', ..."
4,Karen Shaw,Q116502747,https://wikidata.org/wiki/Q116502747,['http://en.wikipedia.org/wiki/Karen_Shaw_Petr...
5,Emma Slowe,Q116502906,https://wikidata.org/wiki/Q116502906,['http://en.wikipedia.org/wiki/Emma_Flower_Tay...
6,Joanna Will,Q116504483,https://wikidata.org/wiki/Q116504483,['http://en.wikipedia.org/wiki/Joanna_Williams...
